
___________
### 0. Delete database

In [3]:
import os
import sqlite3

# Close any existing connections
conn = sqlite3.connect('nursery.db')
conn.close()

# Delete the file completely
if os.path.exists('nursery.db'):
    os.remove('nursery.db')
    print("Database file removed completely!")
else:
    print ("Database file did not exist.")

Database file removed completely!


In [15]:
import sqlite3

# Connect to your database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# To DROP specific tables
try:
    cursor.execute('''
        DROP TABLE IF EXISTS ESPECIE_LOTE
    ''')
    conn.commit()  # Important: Save the changes
    print("Table dropped successfully")
    
except sqlite3.Error as e:
    print(f"Error dropping table: {e}")
    
finally:
    conn.close()  # Always close the connection

Table dropped successfully


______________________
### 1. 
Database creation and table architecture 

In [16]:
import sqlite3

# Connect to database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

try:
    # 1. ESPECIE_LOTE table for static and descriptive attributes of each species, with equivalence between lot and species
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ESPECIE_LOTE (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            NOMBRE_CIENTIFICO TEXT NOT NULL UNIQUE,
            NOMBRE_COMUN TEXT,
            FAMILIA TEXT, 
            FECHA_INICIO DATE,
            GERMINACION_PROMEDIO TEXT CHECK (GERMINACION_PROMEDIO IN ('Menos 8 días', 'De 8 a 10 días', 'De 10 a 15 días', 'Más de 15 días')),
            CRECIMIENTO TEXT CHECK(CRECIMIENTO IN ('Lento', 'Moderado', 'Rápido')),
            RIEGO TEXT CHECK(RIEGO IN ('Bajo', 'Medio', 'Alto')),
            LUZ TEXT CHECK(LUZ IN ('Pleno sol', 'Sombra parcial', 'Sombra')),
            PROPAGACION TEXT,
            NOTAS TEXT,
            FOREIGN KEY (PROPAGACION) REFERENCES METODOS_PROPAGACION(METODO)
        )            
    ''')

    # 2. ENTRADA_PLANTAS Table - Records each planting/addition of plants
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ENTRADA_PLANTAS (
            ID_ENTRADA INTEGER PRIMARY KEY AUTOINCREMENT,
            ESPECIE_LOTE_ID INTEGER NOT NULL,
            CODIGO_ENTRADA TEXT NOT NULL UNIQUE, -- Identificador único: ESPECIE-FECHA-SECUENCIA
            FECHA_ENTRADA DATE NOT NULL,
            TIPO_ENTRADA TEXT CHECK (TIPO_ENTRADA IN ('Trasplante', 'Esqueje', 'Donación', 'Compra')),
            
            -- Fields for seed propagation
            FECHA_SIEMBRA DATE,
            CANTIDAD_SEMILLAS INTEGER,
            FECHA_BROTACION DATE,

            -- Fields for cutting propagation
            FECHA_CORTE DATE,
            CANTIDAD_ESQUEJES INTEGER,
            
            -- General fields
            CANTIDAD_PLANTAS_EMBOLSADAS INTEGER,
            FECHA_EMBOLSADO DATE,
            FASE_LUNAR TEXT CHECK (FASE_LUNAR IN ('Nueva', 'Cuarto Creciente', 'Llena', 'Cuarto Menguante')),
            NOTAS TEXT,
            FOREIGN KEY (ESPECIE_LOTE_ID) REFERENCES ESPECIE_LOTE(ID)
        )
    ''')

    # 3. OBSERVACION_LOTE Table for continuous monitoring
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS OBSERVACION_LOTE(
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            ESPECIE_LOTE_ID INTEGER NOT NULL,
            FECHA_OBSERVACION DATE NOT NULL,
            CANTIDAD_PLANTAS_VIVAS INTEGER NOT NULL,
            ALTURA_PROMEDIO_CM REAL,
            ETAPA_CRECIMIENTO TEXT,
            INDICADOR_SALUD TEXT CHECK(INDICADOR_SALUD IN ('Excelente', 'Buena', 'Regular', 'Mala')),
            NOTAS TEXT,
            FOREIGN KEY (ESPECIE_LOTE_ID) REFERENCES ESPECIE_LOTE(ID),
            FOREIGN KEY (ETAPA_CRECIMIENTO) REFERENCES ETAPA_CRECIMIENTO(ETAPA),
            UNIQUE(ESPECIE_LOTE_ID, FECHA_OBSERVACION)
        )
    ''')

    # 4. REGISTRO_RIEGO table focused on irrigatio
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS REGISTRO_RIEGO (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            ESPECIE_LOTE_ID INTEGER NOT NULL,¿
            FECHA DATE NOT NULL,
            CANTIDAD_AGUA_LT REAL,
            NOTAS TEXT,
            FOREIGN KEY (ESPECIE_LOTE_ID) REFERENCES ESPECIE_LOTE(ID)
        );
    ''')

    # 5. REGISTRO_CLIMA to register rain and weather relevant to the nursery
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS REGISTRO_CLIMA (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            FECHA_HORA_INICIO DATETIME NOT NULL,
            FECHA_HORA_FIN DATETIME NOT NULL,
            CATEGORIA_PRECIPITACION TEXT CHECK (CATEGORIA_PRECIPITACION IN ('Lluvia Intensa', 'Lluvia Moderada', 'Llovizna Persistente')),
            NOTAS TEXT
        )
    ''')

    # 6  REGISTRO_MONITOREO_PLAGAS table for pest control
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS REGISTRO_MONITOREO_PLAGAS (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            ESPECIE_LOTE_ID INTEGER NOT NULL,
            DESCRIPCIÓN_PLAGA TEXT,
            NIVEL_GRAVEDAD TEXT CHECK(NIVEL_GRAVEDAD IN ('Ninguno', 'Bajo', 'Medio', 'Alto')),
            FECHA DATE NOT NULL,
            FASE_LUNAR TEXT CHECK (FASE_LUNAR IN ('Nueva', 'Cuarto Creciente', 'Llena', 'Cuarto Menguante')),
            NOMBRE_PRODUCTO TEXT,   
            DOSIS_PRODUCTO REAL,
            CANTIDAD_AGUA TEXT
        )
    ''')

    # 7 GASTOS table to record operating expenses
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS GASTOS (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            FECHA DATE NOT NULL,
            CATEGORIA TEXT NOT NULL,
            DESCRIPCION TEXT NOT NULL,
            MONTO REAL NOT NULL,
            NOTAS TEXT
        )
    ''')

    conn.commit()
    print("Database schema created successfully!")

except sqlite3.Error as e:
    print(f"An error occurred : {e}")
    conn.rollback()

finally:
    conn.close()

Database schema created successfully!


________
### 2. Backup tables

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# Create METODOS_PROPAGACION table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS METODOS_PROPAGACION (
        METODO TEXT PRIMARY KEY,
        CATEGORIA TEXT,  -- 'simple', 'combinado', 'especial'
        DESCRIPCION TEXT
    )
''')

# Popagation categories
metodos_detallados = [
    ('Semilla', 'simple', 'Propagación sexual por semillas'),
    ('Esqueje', 'simple', 'Propagación asexual por fragmentos de tallo'),
    ('División', 'simple', 'Separación de matas o rizomas'),
    ('Acodo', 'simple', 'Enraizado de ramas sin separar de la planta madre'),
    ('Injerto', 'simple', 'Unión de tejidos de diferentes plantas'),
    ('Semilla-esqueje', 'combinado', 'Especie que se propaga bien por ambos métodos'),
    ('Esqueje-división', 'combinado', 'Común en herbáceas perennes y arbustos'),
    ('Esqueje-acodo', 'combinado', 'Para plantas leñosas de difícil enraizado'),
    ('Todas', 'especial', 'Aplica múltiples métodos eficientemente')
]

In [12]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

# create ETAPA_CRECIMIENTO table with robust structure
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ETAPA_CRECIMIENTO (
        ETAPA TEXT PRIMARY KEY, 
        DESCRIPCION TEXT,
        ORDEN INTEGER UNIQUE, -- Para ordenamiento lógico
        DURACION_ESTIMADA_DIAS INTEGER, -- Duración promedio de la etapa
        REQUIERE_RIEGO_ESPECIAL BOOLEAN DEFAULT 0,
        REQUIERE_FERTILIZACION BOOLEAN DEFAULT 0
    )
''')

# Stage definition relevant for analysis
etapas_vivero = [
    # Etapas iniciales - Pregerminación
    ('RECEPCION_SEMILLA', 'Recepción y Almacenamiento de semillas', 1, 30, 0, 0),
    ('PRE_GERMINACIÓN', 'Tratamientos pre-germinativos', 2, 7, 0, 1),
    ('SIEMBRA', 'Siembra en almácigos o bandejas', 3, 1, 1, 0),

    # Etapas de germinación y plantula
    ('GERMINACION', 'Proceso de germinación activa', 4, 15, 1, 0),
    ('PLANTULA_ALMACIGO', 'Desarrollo inicial', 5, 30, 1, 0),

    # Etapas de crecimiento activo
    ('CRECIMIENTO_VEGETATIVO', 'Crecimiento activo', 6, 45, 1, 1),
    ('PRE_EMBOLSE', 'Plántula lista para el transplante a bolsa', 7, 7, 1, 0),
    ('EMBOLSADO', 'Transplante completado a bolsas', 8, 1, 1, 0),

    # Etapas de endurecimiento y preparación
    ('ACLIMATACION_POST_ EMBOLSE','Dias posteriores al embolse', 9, 14, 1, 1),
    ('ENDURECIMIENTO', 'Aclimatar gradualmente a condiciones exteriores', 10, 21, 1, 1),
    ('LISTA_CAMPO', 'Plata lista para la siembra', 11, 7, 1, 0),

    # Etapas especiales
    ('CUARENTENA', 'Aislamiento por plagas o enfermedades', 12, 30, 1, 0),
    ('RECUPERACION', 'Recuperación post-estrés', 13, 21, 1, 1),
    ('DESCARTE', 'Material para descarte', 14, 1, 0, 0),
]

# Insert nursery stages
cursor.executemany(''' 
    INSERT OR IGNORE INTO ETAPA_CRECIMIENTO
    (ETAPA, DESCRIPCION, ORDEN, DURACION_ESTIMADA_DIAS, REQUIERE_RIEGO_ESPECIAL, REQUIERE_FERTILIZACION)
    VALUES (?, ?, ?, ?, ?, ?)
''',  etapas_vivero)

# Commit changes y cerrar conexión
conn.commit()
conn.close()
